In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
# fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/cori_sam-lib_2022-12-04_01.hdf5"
# fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/2022-12-05_01/sam_lib.hdf5"
fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/2022-12-05_01/sam_lib.hdf5"
data = h5py.File(fname, 'r')
print("Keys:", data.keys())
for kk, vv in data.items():
    print("\t", kk, vv.shape)
print("Attributes:")
for kk, vv in data.attrs.items():
    print("\t", kk, vv)

In [ ]:
import zcode.math as zmath

In [ ]:
fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax()


yy = data['gwb'][:]
# yy = np.median(yy, axis=-1)
ax.plot(xx, np.moveaxis(yy, 1, 0).reshape(yy.shape[1], -1), alpha=0.1, color='k', lw=0.5)

# ff = zmath.argnearest(xx, 1.0)
# yy = data['gwb'][:, ff, :].flatten()
# yy = np.log10(yy)
# kale.dist1d(yy, ax=ax, density=True, carpet=True)

plt.show()

In [ ]:
param_names = data.attrs['param_names']
for ii, par in enumerate(param_names):
    vals = data['params'][:, ii]
    print(f"{ii} {par} : [{vals.min():.2e}, {vals.max():.2e}] {utils.stats(vals)}")

In [ ]:
gwb.max()

In [ ]:
gwb = data['gwb'][:, 0, :].flatten()
print(utils.stats(gwb))
# kale.dist1d(np.log10(gwb))

In [ ]:
kale.corner(data['params'][:].T)